In [ ]:
import pandas as pd
from statsmodels.tsa.api import SimpleExpSmoothing
from datetime import timedelta

# خواندن فایل ورودی
df = pd.read_csv('merged_output2.csv')

# تبدیل DATE_MILADI و HOUR به datetime
df['DATE_MILADI'] = pd.to_datetime(df['DATE_MILADI'])
df['datetime'] = df['DATE_MILADI'] + pd.to_timedelta(df['HOUR'], unit='h')

# مرتب‌سازی بر اساس datetime
df = df.sort_values('datetime').reset_index(drop=True)

# چک کردن duplicate در datetime (برای جلوگیری از خطا)
if df['datetime'].duplicated().any():
    print("هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...")
    # اگر duplicate، میانگین POWER بگیریم (یا sum اگر مناسب‌تر باشه، بسته به داده)
    df = df.groupby('datetime').agg({
        'POWER': 'mean',  # یا 'sum' اگر توان کل باشه
        'DATE_MILADI': 'first',
        'HOUR': 'first',
        'DATE_SHAMSI': 'first',
        'CODE': 'first',
        'UNIT_NO': 'first',
        'DAMA': 'mean',  # یا مناسب‌تر
        'ROTOOBAT': 'mean',
        '12209_G13': 'mean',
        '12210_G13': 'mean',
        'ebraz': 'first',  # یا mean، بسته به معنی
        'importance_factor': 'first'
    }).reset_index()

# استخراج year, month, dayofweek از datetime
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek

# فیلتر داده آموزشی (2022 تا 2023) و تست (از 2024 به بعد)
train_mask = df['year'].between(2022, 2023)
test_mask = df['year'] >= 2024

train = df[train_mask]
test = df[test_mask].copy()

# ایجاد سری زمانی POWER با index datetime
power_series_full = df.set_index('datetime')['POWER']

# تنظیم فرکانس ساعتی با forward fill برای missing hours (اگر داده ناقص باشه)
power_series_full = power_series_full.asfreq('h', method='ffill')  # 'h' به جای 'H'

# اضافه کردن ستون DECLARED به تست
test['DECLARED'] = 0.0

# لوپ روی ردیف‌های تست برای پیش‌بینی سه روز آینده با SES
for idx, row in test.iterrows():
    current_datetime = row['datetime']
    
    if row['ebraz'] == 0:
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # زمان پایه برای آموزش: دقیقاً سه روز قبل از زمان فعلی
    train_end_time = current_datetime - timedelta(days=3)
    
    # داده‌های موجود تا train_end_time
    train_data = power_series_full[power_series_full.index <= train_end_time]
    
    if len(train_data) < 48:  # حداقل داده معقول (مثلاً دو روز)
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # فیت مدل SES روی train_data
    model = SimpleExpSmoothing(train_data).fit()  # alpha بهینه می‌شود
    
    # تعداد گام‌های پیش‌بینی: دقیقاً 72 ساعت (3 روز)
    steps = 72
    
    # پیش‌بینی 72 گام جلو
    forecast = model.forecast(steps)
    
    # مقدار آخرین پیش‌بینی (ساعت مربوطه در سه روز بعد)
    test.at[idx, 'DECLARED'] = forecast.iloc[-1]

# ستون‌های خروجی مورد نظر
output_cols = [
    'HOUR', 'DATE_MILADI', 'DATE_SHAMSI', 'POWER', 'CODE', 'UNIT_NO',
    'DAMA', 'ROTOOBAT', '12209_G13', '12210_G13', 'ebraz', 'importance_factor',
    'year', 'month', 'dayofweek', 'DECLARED'
]

# ذخیره فقط داده‌های تست (2024 به بعد) در فایل خروجی
test[output_cols].to_csv('output_ses.csv', index=False)

print("فایل خروجی output_ses.csv با موفقیت ایجاد شد.")
print(f"تعداد ردیف‌های پیش‌بینی‌شده: {len(test)}")

هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...
